In [1]:
import tensorflow as tf

In [2]:
import tensorflow.keras as keras

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Utils

In [4]:
import array
import collections

from typing import Dict, List, Optional, Text, Tuple

def _create_feature_dict() -> Dict[Text, List[tf.Tensor]]:
  return {"STOCKCODE": [], "RATING": [], "GICS": [], "STOCKNAME": [], "UNIX_TS": []}

def _sample_list(
    feature_lists: Dict[Text, List[tf.Tensor]],
    num_examples_per_list: int,
    random_state: Optional[np.random.RandomState] = None,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Function for sampling a list example from given feature lists."""
  if random_state is None:
    random_state = np.random.RandomState()

  sampled_indices = random_state.choice(
      range(len(feature_lists["STOCKCODE"])),
      size=num_examples_per_list,
      replace=False,
  )
  sampled_STOCKCODE = [
      feature_lists["STOCKCODE"][idx] for idx in sampled_indices
  ]
  sampled_RATING = [
      feature_lists["RATING"][idx]
      for idx in sampled_indices
  ]
  sampled_GICS = [
      feature_lists["GICS"][idx] for idx in sampled_indices
  ]
  sampled_STOCKNAME = [
      feature_lists["STOCKNAME"][idx]
      for idx in sampled_indices
  ]
  sampled_UNIX_TS = [
      feature_lists["UNIX_TS"][idx] for idx in sampled_indices
  ]

  return (
      tf.stack(sampled_STOCKCODE, 0),
      tf.stack(sampled_RATING, 0),
      tf.stack(sampled_GICS, 0),
      tf.stack(sampled_STOCKNAME, 0),
      tf.stack(sampled_UNIX_TS, 0)
  )


def sample_listwise(
    rating_dataset: tf.data.Dataset,
    num_list_per_user: int = 10,
    num_examples_per_list: int = 10,
    seed: Optional[int] = None,
) -> tf.data.Dataset:
  
  random_state = np.random.RandomState(seed)

  example_lists_by_user = collections.defaultdict(_create_feature_dict)

  movie_title_vocab = set()
  for example in rating_dataset:
    user_id = example["CDSACCNO"].numpy()
    example_lists_by_user[user_id]["STOCKCODE"].append(
        example["STOCKCODE"])
    example_lists_by_user[user_id]["RATING"].append(
        example["RATING"])
    example_lists_by_user[user_id]["GICS"].append(
        example["GICS"])
    example_lists_by_user[user_id]["STOCKNAME"].append(
        example["STOCKNAME"])
    example_lists_by_user[user_id]["UNIX_TS"].append(
        example["UNIX_TS"])
    
    movie_title_vocab.add(example["STOCKNAME"].numpy())

    

  tensor_slices = {"CDSACCNO": [], "STOCKCODE": [], "RATING": [], "GICS": [], "STOCKNAME": [], "UNIX_TS": []}

  for user_id, feature_lists in example_lists_by_user.items():
    for _ in range(num_list_per_user):

      # Drop the user if they don't have enough ratings.
      if len(feature_lists["STOCKNAME"]) < num_examples_per_list:
        continue

        '''sampled_STOCKCODE, 0),
      tf.stack(sampled_RATING, 0),
      tf.stack(sampled_GICS, 0),
      tf.stack(sampled_STOCKNAME, 0),
      tf.stack(sampled_UNIX_TS'''

      sampled_STOCKCODE, sampled_RATING, sampled_GICS, sampled_STOCKNAME, sampled_UNIX_TS  = _sample_list(
          feature_lists,
          num_examples_per_list,
          random_state=random_state,
      )
      tensor_slices["CDSACCNO"].append(user_id)
      tensor_slices["STOCKCODE"].append(sampled_STOCKCODE)
      tensor_slices["RATING"].append(sampled_RATING)
      tensor_slices["GICS"].append(sampled_GICS)
      tensor_slices["STOCKNAME"].append(sampled_STOCKNAME)
      tensor_slices["UNIX_TS"].append(sampled_UNIX_TS)

  return tf.data.Dataset.from_tensor_slices(tensor_slices)

# Work

In [5]:
portfolios = tf.data.Dataset.load("../../data/portfolios_tfds_lists")

In [6]:
train_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/train_lists").cache() #data\ratings_train
test_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test_lists").cache()

In [9]:
next(iter(train_ds)), len(train_ds)

({'CDSACCNO': <tf.Tensor: shape=(), dtype=string, numpy=b'RPS-23479-LI/00'>,
  'RATING': <tf.Tensor: shape=(), dtype=float32, numpy=3.0>,
  'STOCKNAME': <tf.Tensor: shape=(), dtype=string, numpy=b'RICHARD PIERIS AND COMPANY PLC'>,
  'GICS': <tf.Tensor: shape=(), dtype=string, numpy=b'Capital Goods'>,
  'STOCKCODE': <tf.Tensor: shape=(), dtype=string, numpy=b'RICH'>,
  'UNIX_TS': <tf.Tensor: shape=(), dtype=float32, numpy=1678213800.0>},
 3077)

In [13]:
train_v1 = sample_listwise(
    train_ds,
    num_list_per_user=50,
    num_examples_per_list=10,
    seed=42
)

test_v1 = sample_listwise(
    test_ds,
    num_list_per_user=1,
    num_examples_per_list=10,
    seed=42
)

In [11]:
next(iter(train_v1))

{'CDSACCNO': <tf.Tensor: shape=(), dtype=string, numpy=b'RPS-23479-LI/00'>,
 'STOCKCODE': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'CIC', b'AMSL', b'RAL', b'REG', b'AEL'], dtype=object)>,
 'RATING': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4., 2., 5., 5., 2.], dtype=float32)>,
 'GICS': <tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'Materials', b'Health Care Equipment & Services',
        b'Food Beverage & Tobacco', b'Consumer Durables & Apparel',
        b'Capital Goods'], dtype=object)>,
 'STOCKNAME': <tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'C I C HOLDINGS PLC', b'ASIRI SURGICAL HOSPITAL PLC',
        b'RENUKA AGRI FOODS PLC', b'REGNIS (LANKA) PLC',
        b'ACCESS ENGINEERING PLC'], dtype=object)>,
 'UNIX_TS': <tf.Tensor: shape=(5,), dtype=float32, numpy=
 array([1.6475418e+09, 1.6681050e+09, 1.7100954e+09, 1.7030970e+09,
        1.6655994e+09], dtype=float32)>}

In [14]:
len(train_v1)

5750

In [15]:
train_v1.save("../../data/rating_lists_10/train")
test_v1.save("../../data/rating_lists_10/test")

In [124]:
next(iter(train_v1.take(1)))['STOCKCODE'].shape[0]

5

In [125]:
# train_v1.save("../../data/train_lists_ds")
# test_v1.save("../../data/test_lists_ds")

In [5]:
train_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/train").cache() #data\ratings_train
test_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test").cache()
portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_tfds").cache()

train_list_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/train_lists_ds").batch(64).cache()
test_list_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test_lists_ds").batch(64).cache()

items_ids = portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
item_names = portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
item_GICS = portfolios.batch(10000).map(lambda x: x["GICS"])

user_ids = portfolios.batch(10000).map(lambda x: x["CDSACCNO"])

unique_item_ids = np.unique(np.concatenate(list(items_ids)))
unique_item_names = np.unique(np.concatenate(list(item_names)))
unique_item_gics = np.unique(np.concatenate(list(item_GICS)))

unique_user_ids = np.unique(np.concatenate(list(user_ids)))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
    from item_embedding import ItemModel
    from user_embedding import UserModel
    
    item_model = ItemModel(
      unique_item_ids = unique_item_ids,
      # unique_item_names = unique_item_names,
      unique_item_gics = unique_item_gics
    )

    user_model = UserModel(
      # use_timestamp = self.use_timestamp,
      unique_user_ids = unique_user_ids, 
      # timestamps = self.timestamps, 
      # timestamp_buckets = self.timestamp_buckets
    )

In [13]:
test_batch = next(iter(train_ds.batch(2)))
test_batch

{'CDSACCNO': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'RPS-51649-LI/00', b'RPS-65961-LI/00'], dtype=object)>,
 'GICS': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'Consumer Services', b'Materials'], dtype=object)>,
 'STOCKCODE': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'EDEN', b'HAYC'], dtype=object)>,
 'RATING': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 0.], dtype=float32)>,
 'STOCKNAME': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'EDEN HOTEL LANKA PLC', b'HAYCARB PLC'], dtype=object)>,
 'UNIX_TS': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.6935066e+09, 1.6661178e+09], dtype=float32)>}

In [14]:
item_inp = (test_batch['STOCKCODE'], test_batch['GICS'])

In [15]:
item_ = item_model(item_inp)

item_

<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
array([[-0.04609909,  0.03363004,  0.02220925,  0.04583086,  0.0479766 ,
         0.00265212, -0.02566367, -0.03968347,  0.00273192,  0.00596799,
        -0.03537735, -0.01152014, -0.02740055, -0.03847308,  0.03074887,
         0.00746394,  0.02557604, -0.03605348,  0.03457588, -0.02633193,
         0.00427165, -0.03910352, -0.0273193 , -0.00017543, -0.03689706,
        -0.04243723, -0.04428256,  0.04085237, -0.0176506 ,  0.0199351 ,
        -0.03132515, -0.03317406],
       [-0.01962997,  0.00674096, -0.02899243,  0.00913846, -0.00501378,
         0.02511619,  0.02637631,  0.00312997, -0.00820818,  0.0129496 ,
        -0.04606619, -0.02202358, -0.00397959,  0.01801481, -0.03398924,
        -0.00290748,  0.02743557, -0.02246892, -0.04797815, -0.02689838,
        -0.03359759, -0.02839024,  0.04539985,  0.00563767,  0.03925189,
         0.00502604, -0.0291852 ,  0.04716368, -0.01551688,  0.01839479,
         0.0248143 ,  0.04978361]], dtyp

In [16]:
user_inp = (test_batch['CDSACCNO'])
user_inp

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'RPS-51649-LI/00', b'RPS-65961-LI/00'], dtype=object)>

In [17]:
(x) = user_inp
x

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'RPS-51649-LI/00', b'RPS-65961-LI/00'], dtype=object)>

In [18]:
user_ = user_model(user_inp)
user_

<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
array([[-0.04318248, -0.02557333, -0.04426644, -0.04888889,  0.02427775,
        -0.04386343,  0.02423866, -0.04564811,  0.01165094,  0.0062684 ,
        -0.02250808,  0.00321429,  0.03284386,  0.02103958,  0.04284323,
        -0.01882521, -0.00179398,  0.02237108, -0.00775488, -0.02413813,
         0.00760498,  0.0143137 ,  0.04713489, -0.03030295,  0.01238028,
         0.04224949, -0.03005378,  0.02673508,  0.0179151 ,  0.04189148,
         0.01856503, -0.03624506],
       [ 0.01269467, -0.01087745,  0.04752555,  0.04394826, -0.02006676,
         0.0152738 , -0.03520433, -0.00353701,  0.04113794, -0.04874184,
         0.00096054,  0.02859712, -0.011659  ,  0.02957839, -0.02672126,
        -0.04440721, -0.03383632, -0.0018695 , -0.01456378,  0.04407836,
         0.00306519,  0.027582  ,  0.02934052,  0.01433052,  0.02391071,
        -0.01924326,  0.00549962, -0.02390081, -0.0490739 ,  0.00514883,
         0.01279065,  0.04338883]], dtyp

In [19]:
list_length = test_batch['STOCKCODE'].shape[1]
list_length

IndexError: tuple index out of range

In [20]:
tf.expand_dims(user_, 1)

<tf.Tensor: shape=(2, 1, 32), dtype=float32, numpy=
array([[[-0.04318248, -0.02557333, -0.04426644, -0.04888889,
          0.02427775, -0.04386343,  0.02423866, -0.04564811,
          0.01165094,  0.0062684 , -0.02250808,  0.00321429,
          0.03284386,  0.02103958,  0.04284323, -0.01882521,
         -0.00179398,  0.02237108, -0.00775488, -0.02413813,
          0.00760498,  0.0143137 ,  0.04713489, -0.03030295,
          0.01238028,  0.04224949, -0.03005378,  0.02673508,
          0.0179151 ,  0.04189148,  0.01856503, -0.03624506]],

       [[ 0.01269467, -0.01087745,  0.04752555,  0.04394826,
         -0.02006676,  0.0152738 , -0.03520433, -0.00353701,
          0.04113794, -0.04874184,  0.00096054,  0.02859712,
         -0.011659  ,  0.02957839, -0.02672126, -0.04440721,
         -0.03383632, -0.0018695 , -0.01456378,  0.04407836,
          0.00306519,  0.027582  ,  0.02934052,  0.01433052,
          0.02391071, -0.01924326,  0.00549962, -0.02390081,
         -0.0490739 ,  0.00514

In [21]:
user_re = tf.repeat(
        tf.expand_dims(user_, 1), [list_length], axis=1)
user_re

NameError: name 'list_length' is not defined

In [137]:
user_re.shape, item_.shape

(TensorShape([2, 5, 32]), TensorShape([2, 5, 16]))

In [138]:
concatenated_embeddings = tf.concat([user_re, item_], 2)
concatenated_embeddings

<tf.Tensor: shape=(2, 5, 48), dtype=float32, numpy=
array([[[-0.0003495 ,  0.00639479,  0.01359694,  0.04848125,
         -0.00087982, -0.03632808, -0.0422799 , -0.04690582,
         -0.03934409, -0.01662402, -0.02959414, -0.04046847,
         -0.04506312, -0.01938169, -0.03839182,  0.04125197,
         -0.00073742,  0.04836239,  0.01136787,  0.04344909,
          0.03735143,  0.01791472,  0.02952674, -0.03846205,
          0.0132929 , -0.04398532, -0.02443273, -0.00138602,
         -0.04856403,  0.00433455, -0.0231949 , -0.04115522,
          0.03979154,  0.03620226, -0.01937484,  0.03311226,
          0.01147978,  0.0093099 , -0.00974836, -0.00482779,
          0.01623065,  0.00183672,  0.00651877, -0.0169461 ,
          0.04624097,  0.00030952,  0.02818774, -0.00379416],
        [-0.0003495 ,  0.00639479,  0.01359694,  0.04848125,
         -0.00087982, -0.03632808, -0.0422799 , -0.04690582,
         -0.03934409, -0.01662402, -0.02959414, -0.04046847,
         -0.04506312, -0.0193816

In [22]:
next(iter(train_ds))

{'CDSACCNO': <tf.Tensor: shape=(), dtype=string, numpy=b'RPS-51649-LI/00'>,
 'GICS': <tf.Tensor: shape=(), dtype=string, numpy=b'Consumer Services'>,
 'STOCKCODE': <tf.Tensor: shape=(), dtype=string, numpy=b'EDEN'>,
 'RATING': <tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
 'STOCKNAME': <tf.Tensor: shape=(), dtype=string, numpy=b'EDEN HOTEL LANKA PLC'>,
 'UNIX_TS': <tf.Tensor: shape=(), dtype=float32, numpy=1693506600.0>}

# Recommender

In [140]:
import tensorflow as tf
import numpy as np
import tensorflow_recommenders as tfrs
import tensorflow_ranking as tfr

from typing import Dict, Text

from item_embedding import ItemModel
from user_embedding import UserModel

def generate_embedding(item, item_model):
  
      item_id = item['STOCKCODE'] 
      item_name = item['STOCKNAME']
      item_gics = item['GICS']

      embedding = item_model([item_id, item_name, item_gics])
      return embedding

class Recommender(tfrs.models.Model):

  def __init__(
    self,
    # use_timestamp,
    portfolios
    ):

    super().__init__()

    # self.use_timestamp = use_timestamp
    self.portfolios = portfolios

    self.items_ids = self.portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
    self.item_names = self.portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
    self.item_GICS = self.portfolios.batch(10000).map(lambda x: x["GICS"])

    self.user_ids = self.portfolios.batch(10000).map(lambda x: x["CDSACCNO"])

    self.unique_item_ids = np.unique(np.concatenate(list(self.items_ids)))
    self.unique_item_names = np.unique(np.concatenate(list(self.item_names)))
    self.unique_item_gics = np.unique(np.concatenate(list(self.item_GICS)))

    self.unique_user_ids = np.unique(np.concatenate(list(self.user_ids)))

    # need these to initialize timestamp embedding layers in future steps

    # self.timestamps = np.concatenate(list(self.portfolios.map(lambda x: x["UNIX_TS"]).batch(100)))

    # self.max_timestamp = self.timestamps.max()
    # self.min_timestamp = self.timestamps.min()

    # self.timestamp_buckets = np.linspace(
    #     self.min_timestamp, self.max_timestamp, num=1000,
    # )

    self.item_model = ItemModel(
      unique_item_ids = self.unique_item_ids,
      unique_item_names = self.unique_item_names,
      unique_item_gics = self.unique_item_gics
    )

    self.user_model = UserModel(
      # use_timestamp = self.use_timestamp,
      unique_user_ids = self.unique_user_ids, 
      # timestamps = self.timestamps, 
      # timestamp_buckets = self.timestamp_buckets
    )

    self.score_model = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])

    self.ranking_task = tfrs.tasks.Ranking(
      loss=tfr.keras.losses.ListMLELoss(),
      metrics=[
        tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
        tf.keras.metrics.RootMeanSquaredError()
      ]
    )

  def call(self, features) -> tf.Tensor:
    user_embeddings = self.user_model(
      (
        features['CDSACCNO'],
        # features['UNIX_TS']
       )
    )

    item_embeddings = self.item_model(
      (
        features['STOCKCODE'],
        # features['STOCKNAME'],
        features['GICS']
      )
    )

    # print('**********',features["STOCKCODE"].shape, '**********')

    list_length = features["STOCKCODE"].shape[1] # changed this to 0. was 1

    print('**********',user_embeddings.shape, '**********')

    user_embedding_repeated = tf.repeat(
        tf.expand_dims(user_embeddings, 1), [list_length], axis=1)

    print('**********',user_embedding_repeated.shape, '**********')
    print('**********',item_embeddings.shape, '**********')

    concatenated_embeddings = tf.concat(
        # [user_embedding_repeated, tf.expand_dims(item_embeddings, 0)], 2)
        [user_embedding_repeated, item_embeddings], 2)

    return self.score_model(concatenated_embeddings)
    
  def compute_loss(self, features, training = False):

    labels = features.pop('RATING')

    scores = self(features)

    return self.ranking_task(
      labels = labels,
      predictions = tf.squeeze(scores, axis = 1)
      )

# Main

In [141]:
train_v2 = train_v1.batch(8)
# next(iter(train_v2))

In [144]:
next(iter(train_v2))

{'CDSACCNO': <tf.Tensor: shape=(8,), dtype=string, numpy=
 array([b'RPS-23479-LI/00', b'RPS-23479-LI/00', b'RPS-23479-LI/00',
        b'RPS-23479-LI/00', b'RPS-23479-LI/00', b'RPS-23479-LI/00',
        b'RPS-23479-LI/00', b'RPS-23479-LI/00'], dtype=object)>,
 'STOCKCODE': <tf.Tensor: shape=(8, 5), dtype=string, numpy=
 array([[b'CIC', b'AMSL', b'RAL', b'REG', b'AEL'],
        [b'RICH', b'TAFL', b'ASIY', b'MHDL', b'HPWR'],
        [b'HAYL', b'CIND', b'DIPD', b'RICH', b'AEL'],
        [b'HEXP', b'DIST', b'TAFL', b'PINS', b'HAYL'],
        [b'DIPD', b'PARQ', b'DIST', b'HEXP', b'HASU'],
        [b'RICH', b'CIND', b'HPWR', b'TJL', b'PARQ'],
        [b'DIAL', b'PINS', b'ALUM', b'DIST', b'KGAL'],
        [b'CIC', b'HPWR', b'DIPD', b'AEL', b'SDB']], dtype=object)>,
 'RATING': <tf.Tensor: shape=(8, 5), dtype=float32, numpy=
 array([[4., 2., 5., 5., 2.],
        [3., 3., 2., 2., 3.],
        [5., 3., 3., 3., 2.],
        [3., 2., 3., 2., 5.],
        [3., 2., 2., 3., 2.],
        [3., 3., 3., 2.

In [145]:
model = Recommender(
    # use_timestamp = True,
    portfolios = portfolios
    )

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

model.fit(
    train_v2, 
    epochs=20, 
    verbose = 1,
    )

Epoch 1/20
********** (1, None, 32) **********
********** (1, 5, None, 32) **********
********** (None, 5, 16) **********


ValueError: in user code:

    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\tensorflow_recommenders\models\base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "C:\Users\naradaw\AppData\Local\Temp\ipykernel_6416\3972399412.py", line 123, in compute_loss
        scores = self(features)
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\utils\traceb